## Regression Modeling
The first part of this notebook follows closely the introductory examples in the Statsmodels online documentation.

We won't try to provide the theoretical basis for regression models here.  You can consult any number of online resources for this, including this explanation of Ordinary Least Squares Regression in wikipedia: https://en.wikipedia.org/wiki/Ordinary_least_squares

We will be using the Statsmodels library for this. Documentation is available here: http://statsmodels.sourceforge.net/stable/index.html

The statistical model is assumed to be $Y = X\beta + \epsilon$, where $\epsilon\sim N\left(0,\sigma^{2}\right)$

We focus here on the simple Ordinary Least Squares (OLS) model that is the most widely used, but makes strong assumptions about the errors being indepentently and identically distributed (i.i.d.).  When these conditions are met, the OLS parameter estimates are the Best Linear Unbiased Estimates (BLUE).

More intuitively (perhaps), what linear regression using the OLS estimator attempts to do is find the vector of parameters $\beta$ such that when you compute a linear function $X\beta$ you generate a predicted array of $\hat{y}$ that, compared to the observed $y$, the squared sum of each observation's error $\epsilon_{i} = \hat{y}_{i} - y_{i}$ across all the observations $\Sigma^{2}\epsilon_{i}$, is minimized.

In [ ]:
import numpy as np
import statsmodels.api as sm
from statsmodels.sandbox.regression.predstd import wls_prediction_std


# Generate artificial data (2 regressors + constant)
nobs = 100
X = np.random.random((nobs, 1))
X = sm.add_constant(X)
beta = [1.0, 3.5]
e = np.random.normal(size=nobs)
y_true = np.dot(X, beta)
y = np.dot(X, beta) + e

# Fit regression model
model = sm.OLS(y, X)
results = model.fit()

# Inspect the results
print(results.summary())


Let's examine the data graphically to better understand it.  First a scatterplot of our X variable against Y.  We skip the Intercept to get the actual X values.

In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt
plt.scatter(X[:,1], y, s=1)
plt.show()

Now we can manually add a plot of the true equation, without the error, and superimpose it.  What OLS is doing is minimizing the sum of all the errors -- the distance from each point to the line, across all the data.

In [ ]:
plt.scatter(X[:,1], y, color = 'r', s = 1)
plt.plot(X[:,1], (beta[0] + beta[1] * X[:,1]), color='b')
plt.show()

In [ ]:
print('Parameters: ', results.params)
print('Standard errors: ', results.bse)
print('Predicted values: ', results.predict())

In [ ]:
print('Parameters: ', results.params)
print('R2: ', results.rsquared)

In [ ]:
prstd, iv_l, iv_u = wls_prediction_std(results)

fig, ax = plt.subplots(figsize=(8,6))

ax.plot(X[:,1], y, 'o', label="data")
ax.plot(X[:,1], y_true, 'b', label="True")
ax.plot(X[:,1], results.fittedvalues, 'r', label="OLS")
ax.plot(X[:,1], iv_u, 'r')
ax.plot(X[:,1], iv_l, 'r')
ax.legend(loc='best');

## Lottery Example

We next use a sample dataset that comes with statsmodels. $y$ is an $N \times 1$ column of data on lottery wagers per capita (Lottery). $X$ is $N \times 7$ with an intercept, the Literacy and Wealth variables, and 4 region binary variables.



In [ ]:
import statsmodels.api as sm
import pandas
from patsy import dmatrices

First we read a standard dataset that comes with Statsmodels.

In [ ]:
df = sm.datasets.get_rdataset("Guerry", "HistData").data
df.head()

In [ ]:
# We don't need all the columns, so let's create a reduced dataframe with only a few colunms in it.
vars = ['Department', 'Lottery', 'Literacy', 'Wealth', 'Region']
df = df[vars]
df.tail()

In [ ]:
# Notice that Region is missing for the last row of data. Let's drop missing values from the dataframe.
df = df.dropna()
df.tail()

In [ ]:
# The traditional way to specify the data for statsmodels regressions is using dmatrices,
# that describe the vector of the dependent variable (y) and the 2D array of independent variables (X)
# Note that this assigns the column to the left of the ~ to y, and the rest, to X
y, X = dmatrices('Lottery ~ Literacy + Wealth + Region', data=df, return_type='dataframe')

In [ ]:
y.head()

In [ ]:
# Notice what happened to the Region variable in the X dmatrix. It automatically created dummy variables
# from a categorical variable, since you cannot use categorical variables directly in a linear regression
X.head()

In [ ]:
mod = sm.OLS(y, X)    # Describe model. This creates a model object but produces no output.
res = mod.fit()       # Fit model. This performs the statistical calculations, but does not output anything.
print(res.summary())   # Summarize model. This prints the already computed model results

In [ ]:
# Notice what it did with the Region variable that initially had 5 unique category values
df['Region'].unique()

### An alternative way to specify models, using R syntax.  This uses the patsy language
See http://patsy.readthedocs.org/en/latest/ for complete documentation

In [ ]:
import statsmodels.formula.api as smf
import numpy as np
import pandas

In [ ]:
mod = smf.ols(formula='Lottery ~ Literacy + Wealth + Region', data=df)
res = mod.fit()
print(res.summary())

### Using Interaction Terms

In [ ]:
mod = smf.ols(formula='Lottery ~ Literacy : Wealth + Region', data=df)
res = mod.fit()
print(res.summary())

In [ ]:
mod = smf.ols(formula='Lottery ~ Literacy * Wealth + Region', data=df)
res = mod.fit()
print(res.summary())

## Now on to Hedonic Regression on Bay Area Housing Sales
We will use a large sample of single family housing sales from the San Francisco Bay Area to estimate a linear regression model in which the dependent variable $y$ is the price of a house at the time of sale, and $X$ is a set of exogenous, or explanatory variables.

What exactly does this give us?  A statistical way to figure out what the component amenities in a house are worth, if you could buy them *a la carte*.  Another way to think of it is, how much do house buyers in the Bay Area during this period pay, on average, for an additional unit of each amenity: square foot of living space, bedroom, bathroom, etc.

In [ ]:
# First, we load a sales transactions file. It has over 200K sales transactions in the Bay Area
import pandas as pd, numpy as np
sales = pd.read_csv('Data/homesales.csv')
sales.rename(columns={ 'SQft' : 'sqft', 'Year_built': 'yrbuilt', 'Lot_size': 'lotsize', 'Sale_price': 'price'}, inplace=True)


print(sales.columns)
sales.drop(['RecordID', 'X', 'Y', 'sales', 'Sale_price_flt', 'parcel_id', '_node_id0', '_node_id1', '_node_id2',], axis=1, inplace=True)
print(sales.columns)
sales.size

In [ ]:
# Now let's load a set of walkscore-like calculations we computed and saved associated with each local street node
# It contains a count of how many of each type of amenity are within a half-kilometer from the nearest node
amenities = pd.read_csv('Data/amenityvalueindicators.csv')
amenities.columns

In [ ]:
# Now merge these two so we have local neighborhood amenities around each house that sold
merged = pd.merge(sales, amenities, left_on='_node_id', right_on='node_id')
merged.head()

In [ ]:
merged.describe()

In [ ]:
# Let's look at some specific key variables
%matplotlib inline
merged['price'].hist(bins=50)

In [ ]:
#merged.plot(kind='scatter',x='sqft',y='price', s=1)
plt.figure(1, figsize=(10,8), )
plt.ylim(0,3000000)
plt.xlim(0, 5000)
plt.scatter(merged['sqft'], merged['price'], s=.01, color='blue')
plt.show()

In [ ]:
#print(cleaned.quantile(.01))
#print(cleaned.quantile(.99))
minprice = merged['price'].quantile(.01)
maxprice = merged['price'].quantile(.99)
minsqft = merged['sqft'].quantile(.01)
maxsqft = merged['sqft'].quantile(.99)
minlot = merged['lotsize'].quantile(.01)
maxlot = merged['lotsize'].quantile(.9)
minyr = 1900
maxyr = 2016

cleaned = merged[(merged['price'] > minprice) & (merged['price'] < maxprice) 
                & (merged['sqft'] > minsqft) & (merged['sqft'] < maxsqft)
                & (merged['yrbuilt'] > 0) & (merged['yrbuilt'] < 2016)
                & (merged['yrbuilt'] > minyr) & (merged['yrbuilt'] < maxyr)
                & (merged['lotsize'] > minlot) & (merged['lotsize'] < maxlot)
                ]

cleaned.describe()

In [ ]:
cleaned['price'].hist(bins=50)

In [ ]:
cleaned['sqft'].hist(bins=50)

In [ ]:
cleaned['lotsize'].hist(bins=50)

In [ ]:
# How much does price correlate with square footage of the house?
#cleaned.plot(kind='scatter',x='sqft',y='price', s=.01, color='g')
plt.figure(1, figsize=(10,8), )
plt.ylim(0,3000000)
plt.xlim(0, 5000)
plt.scatter(cleaned['sqft'], cleaned['price'], s=.01, color='blue')
plt.show()

In [ ]:
# How much does price vary with Lot size?
plt.figure(1, figsize=(10,8), )
plt.ylim(0,3000000)
plt.xlim(0, 10000)
plt.scatter(cleaned['lotsize'], cleaned['price'], s=.01, color='blue')
plt.show()

In [ ]:
# Let's estimate some hedonic regression models, starting simple and adding variables
# We start with price regressed on sqft and an intercept
import statsmodels.api as sm
import statsmodels.formula.api as smf
results = smf.ols('price ~  sqft', data=cleaned).fit()
print(results.summary())

Try interpreting the coefficients of the model above.  What do they mean?

Next we try taking log transformations of the dependent and independent variables.

In [ ]:
# Hedonic regressions are often log-transformed, as semi-log or log-log, with dependent variable log-transformed,
# and some independent variables depending on the data, to improve model fit
results = smf.ols('np.log(price) ~  np.log(sqft)', data=cleaned).fit()
print(results.summary())

In [ ]:
# Let's add lotsize
results = smf.ols('np.log(price) ~  np.log(sqft) + np.log(lotsize)', data=cleaned).fit()
print(results.summary())

In [ ]:
# Let's add a dummy variable or houses built before 1940
results = smf.ols('np.log(price) ~  np.log(sqft) + np.log(lotsize) + yrbuilt<1940',
                 data=cleaned).fit()
print(results.summary())

In [ ]:
# Let's begin exploring how accessibility influences prices
results = smf.ols('np.log(price) ~  np.log(sqft) + np.log(lotsize) + yrbuilt<1940 + shopping',
                 data=cleaned).fit()
print(results.summary())

In [ ]:
#cleaned.loc[:, 'sanfran'] = cleaned['City']=='San Francisco'
cleaned['sanfran'] = False
cleaned['sanfran'] = cleaned['City']=='San Francisco'
results = smf.ols('np.log(price) ~  np.log(sqft) + np.log(lotsize) + yrbuilt<1940 + shopping + sanfran', 
                 data=cleaned).fit()
print(results.summary())
print('Number of San Francisco sales: ', cleaned['sanfran'].sum())

In [ ]:
errors =  (results.predict() - np.log(cleaned['price'])) / np.log(cleaned['price'])
errors.describe()

In [ ]:
plt.figure(1, figsize=(10,8), )
plt.ylim(11.5, 14.5)
plt.xlim(11.5, 14.5)
plt.scatter(np.log(cleaned['price']), results.predict(), s=.01, color='green')
plt.show()

In [ ]:
plt.figure(1, figsize=(10,8), )
plt.scatter(np.log(cleaned['price']), errors, s=.01, color='red')
plt.show()

In [ ]:
cleaned_low = cleaned[cleaned['price'] < 600000]

In [ ]:
results_low = smf.ols('np.log(price) ~  np.log(sqft) + np.log(lotsize) + yrbuilt<1940 + shopping + sanfran', 
                 data=cleaned_low).fit()
print(results_low.summary())

In [ ]:
cleaned_high = cleaned[cleaned['price'] > 600000]

In [ ]:
results_high = smf.ols('np.log(price) ~  np.log(sqft) + np.log(lotsize) + yrbuilt<1940 + shopping + sanfran', 
                 data=cleaned_high).fit()
print(results_high.summary())

In [ ]:
results_lin = smf.ols('price ~  sqft + lotsize + yrbuilt<1940 + shopping + sanfran', 
                 data=cleaned).fit()
print(results_lin.summary())

In [ ]:
plt.figure(1, figsize=(10,8), )
plt.ylim(200000, 2000000)
plt.xlim(200000, 2000000)
plt.scatter(cleaned['price'], results_lin.predict(), s=.01, color='green')
plt.show()

###  Below is an experiment with Robust Linear Models in Statsmodels

See the docs for more background on this and related approaches:
http://statsmodels.sourceforge.net/devel/examples/notebooks/generated/robust_models_0.html

In [ ]:
y, X = dmatrices('price ~  sqft + lotsize + yrbuilt<1940 + shopping + sanfran', data=cleaned, return_type='dataframe')
huber_t = sm.RLM(y, X, M=sm.robust.norms.HuberT())
hub_results = huber_t.fit()
print('Parameters \n')
print(hub_results.params)
print()
print('Standard Errors \n')
print(hub_results.bse)
print(hub_results.summary(yname='y',
            xname=['var_%d' % i for i in range(len(hub_results.params))]))

## Your turn

* Experiment with the regression model to see how much you can improve this regression model and its fit to the data.  

* If you want to learn more about robust regression, experiment with it and try to compare its prediction errors with that of OLS for rhe same model specification.

* Try loading the rentals listing data and census data we used in the last session, merge them, and set up a hedonic regression on rents in the Bay Area.